In [162]:
from sklearn import *
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.ensemble import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score

In [163]:
training=pd.read_csv("kernel1training.csv")
testing=pd.read_csv("kernel1testing.csv")

In [164]:
training.columns

Index(['air_store_id', 'visit_date', 'visitors', 'dow', 'year', 'month',
       'holiday_flg', 'min_visitors', 'mean_visitors', 'median_visitors',
       'max_visitors', 'count_observations', 'air_genre_name', 'air_area_name',
       'latitude', 'longitude', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y',
       'rv1_y', 'rs2_y', 'rv2_y', 'id', 'total_reserv_sum',
       'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int',
       'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id2'],
      dtype='object')

In [165]:
weather=pd.read_csv("data/weather2.csv")

In [166]:
weather.head()

,Unnamed: 0,STATION,PRCP,SNWD,TAVG,TMAX,TMIN
0,0,JAM00047918,0.5,NaN,22.4,NaN,17.7
1,1,JAM00047918,0.0,NaN,24.2,26.5,NaN
2,2,JAM00047918,4.1,NaN,21.7,23.6,NaN
3,3,JAM00047918,9.9,NaN,22.6,NaN,20.6
4,4,JAM00047918,36.1,NaN,22.1,25.2,NaN


In [167]:
air_store_weather_station= pd.read_csv("data/air_store_info_with_station.csv")

In [168]:
air_store_weather_station.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,station_id,station_name
0,air_0f0cdeee6c9bf3d7,Italian/French,Hy?go-ken K?be-shi Kumoid?ri,34.695124,135.197852,JA000047770,"KOBE, JA"
1,air_7cc17a324ae5c7dc,Italian/French,Hy?go-ken K?be-shi Kumoid?ri,34.695124,135.197852,JA000047770,"KOBE, JA"
2,air_fee8dcf4d619598e,Italian/French,Hy?go-ken K?be-shi Kumoid?ri,34.695124,135.197852,JA000047770,"KOBE, JA"
3,air_a17f0778617c76e2,Italian/French,Hy?go-ken K?be-shi Kumoid?ri,34.695124,135.197852,JA000047770,"KOBE, JA"
4,air_83db5aff8f50478e,Italian/French,T?ky?-to Minato-ku Shibak?en,35.658068,139.751599,JA000047662,"TOKYO, JA"


In [169]:
training=pd.merge(training,air_store_weather_station[["air_store_id","station_id"]], on="air_store_id")

In [170]:
training.columns

Index(['air_store_id', 'visit_date', 'visitors', 'dow', 'year', 'month',
       'holiday_flg', 'min_visitors', 'mean_visitors', 'median_visitors',
       'max_visitors', 'count_observations', 'air_genre_name', 'air_area_name',
       'latitude', 'longitude', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y',
       'rv1_y', 'rs2_y', 'rv2_y', 'id', 'total_reserv_sum',
       'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int',
       'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id2',
       'station_id'],
      dtype='object')

In [173]:
training=pd.merge(training, weather,left_on=["station_id"],right_on=["STATION"])

MemoryError: 

In [ ]:
train_X=training.drop(["visitors","air_store_id","id","visit_date"],axis=1)
train_Y=training["visitors"]
test_X=testing.drop(["visitors","air_store_id","id","visit_date"],axis=1)

In [ ]:
train_X.dtypes

In [ ]:
br=BaggingRegressor()

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(train_X, train_Y, test_size=0.4, random_state=0)

In [ ]:
len(X_train)

In [ ]:
br.fit(X_train, np.log1p(y_train.values))

In [ ]:
y1=br.predict(X_test)

In [ ]:
rmse=metrics.mean_squared_error(np.log1p(y_test.values), y1)**0.5

In [ ]:
rmse

In [ ]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

In [ ]:
model1.fit(X_train,np.log1p(y_train.values))

In [ ]:
y_p1=model1.predict(X_test)

In [ ]:
rmse1=metrics.mean_squared_error(np.log1p(y_test.values), y_p1)**0.5

In [ ]:
rmse1

In [ ]:
model2.fit(X_train,np.log1p(y_train.values))

In [ ]:
y_p2=model2.predict(X_test)
rmse2=metrics.mean_squared_error(np.log1p(y_test.values), y_p2)**0.5

In [ ]:
rmse2

In [ ]:
rfregressor=RandomForestRegressor()

In [ ]:
rfregressor.fit(X_train,np.log1p(y_train.values))

In [ ]:
y_p3=rfregressor.predict(X_test)
rmse3=metrics.mean_squared_error(np.log1p(y_test.values), y_p3)**0.5
rmse3

In [ ]:
def rmle(y1,y2):
    return metrics.mean_squared_log_error(y1,y2)**0.5

In [ ]:
scores=cross_val_score(rfregressor, train_X,train_Y, cv=5,scoring=metrics.make_scorer(rmle) )

In [ ]:
scores.mean()

In [ ]:
rfregressor.fit(train_X,np.log1p(train_Y.values))

In [ ]:
test['visitors'] = (model1.predict(test[col]) + model2.predict(test[col])) / 2
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()

In [ ]:
testing["visitors"]=rfregressor.predict(test_X)

In [ ]:
testing['visitors'] = np.expm1(testing['visitors']).clip(lower=0.)

In [ ]:
submission = testing[['id','visitors']].copy()

In [ ]:
submission.to_csv("rfsubmission.csv",index=False)

In [ ]:
rfregressor=RandomForestRegressor(max_features=None,min_samples_leaf=10, random_state=100,verbose=1)

In [ ]:
scores=cross_val_score(rfregressor, train_X,train_Y, cv=5,scoring=metrics.make_scorer(rmle) )

In [ ]:
scores.mean()

In [ ]:
metrics.roc_auc_score(y,model.oob_prediction)

In [ ]:
sample_leaf_options = [1,5,10,50,100,200,500]
estimator_optins=[10,100,500]
for es in estimator_optins:
    rfregressor=RandomForestRegressor(n_estimators=es,max_features="sqrt",min_samples_leaf=10, random_state=100)
    scores=cross_val_score(rfregressor, train_X,train_Y, cv=5,scoring=metrics.make_scorer(rmle) )
    print(scores.mean())

In [ ]:
len(train_X)

In [ ]:
knnr= neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

In [ ]:
sqrt:0.53124316866479737 auto: 0.53195300522562494 None: 0.53195300522562494